# [Advent of Code 2020 Day 13](https://adventofcode.com/2020/day/13)

More simulation perhaps? The `x`s scare me though. I am going to guess part 2 will involve simulating with bus lines would be optimal vs. part 1.

## Initial setup

In [20]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [21]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    earliest_time = int(next(gen))
    ids = next(gen).split(",")

    return earliest_time, ids

## Part 1
Just a basic little brute-force algorithm... this surely won't come back to bite me in the ass... hehe...

In [22]:
def part_one(data: Any) -> int | str:
    earliest_time, ids = data
    best_time = math.inf
    best_bus = math.inf
    for bus_id in ids:
        if bus_id == "x":
            continue
        bus = int(bus_id)
        curr_time = 0
        while curr_time < earliest_time:
            curr_time += bus
        if curr_time < best_time:
            best_time = curr_time
            best_bus = bus
    return (best_time - earliest_time) * best_bus

In [23]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 295
    assert part_one(parse_input("input")) == 138

.                                                                                            [100%]
1 passed in 0.02s


## Part 2
I feel like there's some cool modular arithmetic solution to this... but idk. Maybe Chinese Remainder Theorem if anything.

In [24]:
def part_two_preliminary(data: Any) -> int | str:
    ids = data[1]  # we no longer need the first parameter

    t = 0
    displacement = {}

    for bus_id in ids:
        if bus_id == "x":
            # Blank branch because either way we want to increment
            pass
        else:
            displacement[int(bus_id)] = t
        t += 1

    timing = {key: 0 for key in sorted(displacement.keys())}
    numeric_ids = [int(num) for num in ids if num.isdecimal()]

    iterations = 0
    while True and iterations < 100:
        iterations += 1
        last_timestamp = timing[numeric_ids[0]]
        print(timing)
        solved = True
        for i in range(1, len(numeric_ids)):
            if timing[numeric_ids[i]] - last_timestamp != displacement[numeric_ids[i]]:
                solved = False
                break
        if solved:
            return timing[numeric_ids[0]]
        for bus_id in timing.keys():
            timing[bus_id] += bus_id

In [25]:
%%ipytest
def test_part_two_preliminary():
    assert part_two_preliminary(parse_input("example1")) == 1068781
    #assert part_two_preliminary(parse_input("input")) == 0x3f3f3f3f + 2

F                                                                                            [100%]
============================================ FAILURES =============================================
____________________________________ test_part_two_preliminary ____________________________________

    def test_part_two_preliminary():
>       assert part_two_preliminary(parse_input("example1")) == 1068781
E       AssertionError: assert None == 1068781
E        +  where None = part_two_preliminary((939, ['7', '13', 'x', 'x', '59', 'x', ...]))
E        +    where (939, ['7', '13', 'x', 'x', '59', 'x', ...]) = parse_input('example1')

C:\Users\Jason\AppData\Local\Temp\ipykernel_44504\3683703070.py:2: AssertionError
-------------------------------------- Captured stdout call ---------------------------------------
{7: 0, 13: 0, 19: 0, 31: 0, 59: 0}
{7: 7, 13: 13, 19: 19, 31: 31, 59: 59}
{7: 14, 13: 26, 19: 38, 31: 62, 59: 118}
{7: 21, 13: 39, 19: 57, 31: 93, 59: 177}
{7: 28, 13: 52, 19: 7

Yeah, this isn't going anywhere. I'm going to have to rethink this... first, I think I'm going to make a function that determines if a certain bus can stop at a certain timestamp.

How about this: find the largest bus ID (in the example, it's 59). Because it's the largest, we know that it will be the weakest link when it comes to "aligning" at a certain time (a bus ID, 2, in another case, would satisfy every other timestamp - but 59 is an extremely fickle number and would only satisfy every 59 timestamps). Then, at every step of the highest bus ID, we check whether every other bus can exist at their offsets. This would involve "reshaping" the displacement array as follows.

At first, we see the following displacements:
- Bus 7 would depart at some arbitrary timestamp `t`
- Bus 13 departs at some timestamp `t + 1`
- Bus 59 would depart at some timestamp `t + 4` (because there are two `x`s between 13 and 59 that have no restrictions)
- Bus 31 would depart at some timestamp `t + 6` (because there is a single `x` before 31)
- Bus 19 would depart at some timestamp `t + 7`

Once we determine bus ID 59 to be the desired bus to "jump", we would re-center the displacement array as:
- Bus 7 would depart at some arbitrary timestamp `t - 4`
- Bus 13 would depart at some timestamp `t - 3`
- Bus 59 would depart at some timestamp `t`
- Bus 31 would depart at some timestamp `t + 2`
- Bus 19 would depart at some timestamp `t + 3`

Starting off at timestamp `t`, we check the modified displacement array. So, we check:
- For bus 7, whether `t - 4 % 7 == 0`
- For bus 13, whether `t - 3 % 13 == 0`
- ...
- For bus 19, whether `t + 3 % 19 == 0`

Then if it returns true for all of them, then we return the answer. Otherwise, we continually add 59 to `t`, because we know that's the weakest link.

This is still somewhat brute force, but it'll definitely get my head in the right mindset if I need to apply Chinese Remainder Theorem after. Let's make the function!

In [26]:
def normalize_displacement(displacement: dict[int, int]) -> dict[int, int]:
    """
    Takes a displacement array and returns the biggest bus ID as well as a dictionary that's normalized w.r.t. the biggest bus ID.
    :param displacement: dictionary containing displacements for bus IDs. Bus ID b is keyed to an integer value d, such that bus b must depart at timestamp t + d
    :return: a tuple containing the maximum element, and the normalized displacement dictionary
    """
    new_displacement = {key: value for key, value in displacement.items()}
    largest_bus_id = max(new_displacement.keys())
    largest_bus_id_original_displacement = new_displacement[largest_bus_id]

    for key in new_displacement.keys():
        new_displacement[key] -= largest_bus_id_original_displacement

    return new_displacement

In [27]:
%%ipytest
def test_normalize_displacement():
    assert normalize_displacement({7: 0, 13: 1, 59: 4, 31: 6, 19: 7}) == {7: -4, 13: -3, 59: 0, 31: 2, 19: 3}

.                                                                                            [100%]
1 passed in 0.01s


Now to make the actual function.

In [28]:
def part_two_modular(data: Any) -> int | str:
    ids: list[int | str] = data[1]  # we no longer need the first parameter

    t: int = 0
    displacement: dict[int, int] = {}

    for bus_id in ids:
        if bus_id == "x":
            # Blank branch because either way we want to increment
            pass
        else:
            displacement[int(bus_id)] = t
        t += 1

    smallest_id: int = min(displacement.keys())
    largest_id: int = max(displacement.keys())

    normalized_displacement: dict[int, int] = normalize_displacement(displacement)

    numeric_ids: list[int] = sorted([int(num) for num in ids if num.isdecimal()])

    t = 0

    while True:
        solved = True
        for bus_id in numeric_ids:
            if (t + normalized_displacement[bus_id]) % bus_id != 0:
                solved = False
                break
        if solved:
            return t + normalized_displacement[smallest_id]
        t += largest_id

In [29]:
%%ipytest
def test_part_two_modular():
    assert part_two_modular(parse_input("example1")) == 1068781
    # assert part_two_modular(parse_input("input")) == 0x3f3f3f3f + 2

.                                                                                            [100%]
1 passed in 0.01s


Alas... it was as I expected. I was able to solve the example case, but I knew for a fact that I'd TLE for the actual input. I mean, for such a small input, 1068781 is a huge result.

### The Chinese Remainder Theorem
Those who have taken CIS\*4520 Introduction to Cryptography at the University of Guelph will know this well. :)

It states, according to Wikipedia:
> If one knows the remainders of the Euclidean division of an integer $n$ by several integers, then one can determine, _uniquely_, the remainder of the division of $n$ by the product of these integers, under the condition that the divisors are _pairwise co-prime_ (i.e., no two divisors share a common factor other than 1).

> The Chinese Remainder Theorem is widely used for computing with large integers, as it allows **replacing a computation for which one knows a bound on the size of the result by several similar computations on small integers**

Nice! Thanks Obimbo.

An example of this problem is the linear system:
$x \equiv 3\ (mod\ 5)$
$x \equiv 1\ (mod\ 7)$
$x \equiv 6\ (mod\ 8)$

Here we are looking for, respectively, the value of 3 in the system of integers modulo 5 (in this system we map an arbitrary integer $n$ into the system of integers modulo 5 by applying the mod operation), the value of 1 in the system of integers mod 7, and the value of 6 in the system of integers mod 8, mapped from a single value $x$.

For this example in particular, the answer is 78, because:
$78\ mod\ 5 \equiv 3$
$78\ mod\ 7 \equiv 7$
$78\ mod\ 6 \equiv 8$

The answer 78 is given in the system of integers modulo $(5*7*8)$ or $280$.

Looking back at our example:
- Bus 7 would depart at some arbitrary timestamp `t`
- Bus 13 departs at some timestamp `t + 1`
- Bus 59 would depart at some timestamp `t + 4` (because there are two `x`s between 13 and 59 that have no restrictions)
- Bus 31 would depart at some timestamp `t + 6` (because there is a single `x` before 31)
- Bus 19 would depart at some timestamp `t + 7`

We can actually model this in the form of the Chinese Remainder Theorem:
$t \equiv 0\ (mod\ 7)$
$t \equiv 1\ (mod\ 13)$
$t \equiv 4\ (mod\ 59)$
$t \equiv 6\ (mod\ 31)$
$t \equiv 7\ (mod\ 19)$
because we don't actually care about the value of $t$ - we care about the displacements; in other words - the remainder! And each bus exists in the system of integers modulo its ID; so we are looking for timestamp $t$ for bus 7 such that it is value 0 in the system of integers modulo 7; and this $t$ must also be the value 1 in the system of integers modulo 13 for bus 13, etc... and the answer is $1068788\ (mod\ 3162341)$.

...it all makes sense now. But how do we codify this?

(...this is the part where I watch Neso Academy and hope things work out...)

OK, I don't understand why it works, but I understand the algorithm from [here](https://www.math.cmu.edu/~mradclif/teaching/127S19/Notes/ChineseRemainderTheorem.pdf). Time to bullshit a function copying it. First I'll make a function that implements the modular multiplicative inverse (in a brute-force manner, just so I can wrap my mind around this whole CRT thing).

In [30]:
def mod_inverse(value: int, modulo: int) -> int:
    attempt: int = value
    times: int = 1
    while attempt % modulo != 1:
        times += 1
        attempt += value
    return times

In [31]:
%%ipytest
def test_mod_inverse():
    assert mod_inverse(77, 5) == 3
    assert mod_inverse(55, 7) == 6
    assert mod_inverse(35, 11) == 6

.                                                                                            [100%]
1 passed in 0.01s


In [32]:
def crt(rules: list[tuple[int, int]]) -> int:
    """
    Takes multiple rules for i 1..n of the form (a_i, b_i) such that each rule forms:
      x = a_1 (mod b_1)
      x = a_2 (mod b_2)
      ...
      x = a_n (mod b_n)
    :param rules: an array of rules of the form (a, b) for CRT
    :return: the answer to CRT, modulo (b_1 * b_2 * ... * b_n)
    """
    answer_mod: int = 1
    for value, modulo in rules:
        answer_mod *= modulo

    bezouts: list[int] = []
    for value, modulo in rules:
        bezouts.append(answer_mod // modulo)

    inverses: list[int] = []
    for i in range(len(rules)):
        inverses.append(mod_inverse(bezouts[i], rules[i][1]))

    answer = 0

    for i in range(len(rules)):
        answer += inverses[i] * rules[i][0] * bezouts[i]

    return answer % answer_mod

In [33]:
%%ipytest
def test_crt():
    assert crt(
        [
            (2, 5),
            (3, 7),
            (10, 11),
        ]
    ) == 87 # (mod 385)
    assert crt(
        [
            (2, 5),
            (3, 7),
        ]
    ) == 17 # (mod 35)
    assert crt(
        [
            (0, 7),
            (1, 13),
            (4, 59),
            (6, 31),
            (7, 19),
        ]
    ) == 1068788 # (mod 3162341)

F                                                                                            [100%]
============================================ FAILURES =============================================
____________________________________________ test_crt _____________________________________________

    def test_crt():
        assert crt(
            [
                (2, 5),
                (3, 7),
                (10, 11),
            ]
        ) == 87 # (mod 385)
        assert crt(
            [
                (2, 5),
                (3, 7),
            ]
        ) == 17 # (mod 35)
>       assert crt(
            [
                (0, 7),
                (1, 13),
                (4, 59),
                (6, 31),
                (7, 19),
            ]
        ) == 1068788 # (mod 3162341)
E       assert 2093560 == 1068788
E        +  where 2093560 = crt([(0, 7), (1, 13), (4, 59), (6, 31), (7, 19)])

C:\Users\Jason\AppData\Local\Temp\ipykernel_44504\2175785668.py:15: AssertionError
=

Hmm... that's strange. The CRT examples work, but when I use the AoC example input it completely flops. My answer was $2093560$ when AoC expected $1068788$. Let's see why:
$2093560\ mod\ 7 = 0$
$2093560\ mod\ 13 = 1$
$2093560\ mod\ 59 = 4$
$2093560\ mod\ 31 = 6$
$2093560\ mod\ 19 = 7$
now let's look at AoC's answer:
$1068788\ mod\ 7 = 0$
$1068788\ mod\ 13 = 6$
$1068788\ mod\ 59 = 3$
$1068788\ mod\ 31 = 1$
$1068788\ mod\ 19 = 0$
unfortunately I can't find any immediate pattern... except!
$1068788 + 0\ mod\ 7 = 0$
$1068788 + 1\ mod\ 13 = 0$
$1068788 + 4\ mod\ 59 = 0$
$1068788 + 6\ mod\ 31 = 0$
$1068788 + 7\ mod\ 19 = 0$
it appears I forgot to displace the numbers from their respective starts properly. But how do I do this?

...wait I'm an idiot...the function is correct (I verified the output 2093560 online in a CRT calculator, and it's correct)...I was misinterpreting the results.

You see, the numbers go backwards. I was trying to add to get the modulo to return 0, when in reality I should have been subtracting.
$(2093560 - 0)\ mod\ 7 = 0$
$(2093560 - 1)\ mod\ 13 = 0$
$(2093560 - 4)\ mod\ 59 = 0$
$(2093560 - 6)\ mod\ 31 = 0$
$(2093560 - 7)\ mod\ 19 = 0$

This means the answer to the AoC is not actually CRT, but the subtraction of it from the modulus. The answer is modulo $31623414$, and $3162341-2093560=1068781$ which is the answer all along.

In [34]:
def crt_but_built_different(rules: list[tuple[int, int]]) -> int:
    """
    Takes multiple rules for i 1..n of the form (a_i, b_i) such that each rule forms:
      x = a_1 (mod b_1)
      x = a_2 (mod b_2)
      ...
      x = a_n (mod b_n)
    :param rules: an array of rules of the form (a, b) for CRT
    :return: the answer to CRT, modulo (b_1 * b_2 * ... * b_n)
    """
    answer_mod: int = 1
    for value, modulo in rules:
        answer_mod *= modulo

    bezouts: list[int] = []
    for value, modulo in rules:
        bezouts.append(answer_mod // modulo)

    inverses: list[int] = []
    for i in range(len(rules)):
        inverses.append(mod_inverse(bezouts[i], rules[i][1]))

    answer = 0

    for i in range(len(rules)):
        answer += inverses[i] * rules[i][0] * bezouts[i]

    return answer_mod - (answer % answer_mod)

In [35]:
%%ipytest
def test_crt_but_built_different():
    assert crt_but_built_different(
        [
            (0, 7),
            (1, 13),
            (4, 59),
            (6, 31),
            (7, 19),
        ]
    ) == 1068781 # (mod 3162341)

.                                                                                            [100%]
1 passed in 0.01s


In [36]:
def part_two_but_built_different(data: Any) -> int | str:
    ids: list[int | str] = data[1]  # we no longer need the first parameter

    t: int = 0
    displacement: dict[int, int] = {}

    for bus_id in ids:
        if bus_id == "x":
            # Blank branch because either way we want to increment
            pass
        else:
            displacement[int(bus_id)] = t
        t += 1

    rules = [(modulo, value) for value, modulo in displacement.items()]

    return crt_but_built_different(rules)

OK, I'm feeling pretty confident about this. Luckily the page also gives 5 other examples. Let's see if that works:

In [37]:
%%ipytest
def test_part_two_but_built_different():
    assert part_two_but_built_different(parse_input("example1")) == 1068781
    assert part_two_but_built_different((None, "17,x,13,19".split(","))) == 3417
    assert part_two_but_built_different((None, "67,7,59,61".split(","))) == 754018
    assert part_two_but_built_different((None, "67,x,7,59,61".split(","))) == 779210
    assert part_two_but_built_different((None, "67,7,x,59,61".split(","))) == 1261476
    assert part_two_but_built_different((None, "1789,37,47,1889".split(","))) == 1202161486

.                                                                                            [100%]
1 passed in 0.01s


Hooooooooh boy I'm excited. I think this might be it.

In [38]:
%%ipytest
def test_the_ultimate_input():
    assert part_two_but_built_different(parse_input("input")) == 226845233210288

.                                                                                            [100%]
1 passed in 0.00s


LFG!!!!! WOOOOOOO

It appears I made the same mistake as `danvk` [on GitHub](https://github.com/danvk/aoc2020):

> but the solution was 1068781. I noticed that this was close to the difference of those two numbers, so I tried it… and 3162341 - 2093560 = 1068781! So I tried this on the big problem and it worked.
>
> Looking back at this, I had the congruences messed up. If you want bus 19 to show up one timestamp after bus 7, then you need n + 1 = 0 (mod 19), not n = 1 (mod 19). That explains why I had the answer exactly backwards! My solution did work, I was just solving the wrong problem. In retrospect, writing more tests on small inputs would have helped me find this.
>
> There are very efficient ways to calculate the multiplicative inverse of a number mod a prime, but my brute force solution worked fine in practice.
>
> Apparently this problem is just the Chinese Remainder Theorem. One other wrinkle that tripped me up: because of the way the problem is constructed, sometimes the residue was larger than the prime. So you have to do some addition / modulus in the problem setup, too.